# 0. 준비운동
- 필요한 라이브러리들을 불러옵니다.
- 각각의 클래스의 label(`:int`)을 지정합니다.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
import torch
from typing import Dict, Iterable, List, Any, Tuple
import pandas as pd
from tqdm import tqdm
from random import randint
from sklearn.metrics import f1_score, accuracy_score, auc, roc_curve
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
CLS2IDX = {
    'no_relation': 0,
    'org:member_of': 1,
    'org:top_members/employees': 2,
    'org:alternate_names': 3,
    'per:date_of_birth': 4,
    'org:place_of_headquarters': 5,
    'per:employee_of': 6,
    'per:origin': 7,
    'per:title': 8,
    'org:members': 9,
    'per:schools_attended': 10,
    'per:colleagues': 11,
    'per:alternate_names': 12,
    'per:spouse': 13,
    'org:founded_by': 14,
    'org:political/religious_affiliation': 15,
    'per:children': 16,
    'org:founded': 17,
    'org:number_of_employees/members': 18,
    'per:place_of_birth': 19,
    'org:dissolved': 20,
    'per:parents': 21,
    'per:religion': 22,
    'per:date_of_death': 23,
    'per:place_of_residence': 24,
    'per:other_family': 25,
    'org:product': 26,
    'per:siblings': 27,
    'per:product': 28,
    'per:place_of_death': 29,
}
IDX2CLS = {
    0: 'no_relation',
    1: 'org:member_of',
    2: 'org:top_members/employees',
    3: 'org:alternate_names',
    4: 'per:date_of_birth',
    5: 'org:place_of_headquarters',
    6: 'per:employee_of',
    7: 'per:origin',
    8: 'per:title',
    9: 'org:members',
    10: 'per:schools_attended',
    11: 'per:colleagues',
    12: 'per:alternate_names',
    13: 'per:spouse',               # obj, sbj 스왑 가능
    14: 'org:founded_by',
    15: 'org:political/religious_affiliation',
    16: 'per:children',
    17: 'org:founded',
    18: 'org:number_of_employees/members',
    19: 'per:place_of_birth',
    20: 'org:dissolved',
    21: 'per:parents',              #<-> children
    22: 'per:religion',
    23: 'per:date_of_death',
    24: 'per:place_of_residence',
    25: 'per:other_family',
    26: 'org:product',
    27: 'per:siblings',
    28: 'per:product',
    29: 'per:place_of_death'
}    

2021-10-05 07:06:48.519367: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


cuda


# 1. Model과 Tokenizer 설정하기

- 허깅페이스 모델 허브에서 가져올 MODEL_NAME을 지정합니다.
- model의 구조를 파악해봅니다.
    - 특히 마지막 classifer 레이어의 구조를 우리가 원하는 클래시피케이션에 맞게 조정해야합니다.

In [2]:
MODEL_NAME =  'kykim/bert-kor-base' #"ainize/klue-bert-base-mrc"
tokenizer = AutoTokenizer.from_pretrained( MODEL_NAME )
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
model

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

- classifier를 우리의 task에 맞게 조정해줍시다.
    - `nn.Linear(768, 2, bias=True) -> nn.Linear(768, 30, bias=True)`

In [3]:
# 위의 구조에 맞춰 파인튜닝해줍시다.
model.classifier = torch.nn.Linear(768, 30, bias=True)

# 2. 데이터 

- **df** `(DataFrame)` :
    - **train.csv**파일을 데이터프레임으로 불러온 것
- **df_dict** `(Dict[int, DataFrame])` :
    - **df**를 각각의 label별로 분류한 딕셔너리
- **df_dict_len** `(Dict[int, int])` :
    - 각각의 label별 데이터의 개수(row의 개수)를 분류한 딕셔너리

In [4]:
df = pd.read_csv('../dataset/train/train.csv')
df['subject_entity'] = df['subject_entity'].map(lambda x: eval(x)['word'])
df['object_entity'] = df['object_entity'].map(lambda x: eval(x)['word'])
df['label'] = df['label'].map(lambda x : CLS2IDX[x])
df_dict = { i : df.loc[ df["label"] == i , : ].reset_index(drop=True) for i in range(30)}
df_dict_len = { i : len(df_dict[i]) for i in range(30)}

In [5]:
df_dict[11] #label이 11인 데이터만 가져온다 

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,김종필,박정희,11,wikipedia
1,157,"아나킨 스카이워커는 미디클로리언 수치가 20000이 넘는, 선천적으로 가진 포스의 ...",아나킨 스카이워커,콰이곤 진,11,wikipedia
2,276,《Riding with the King》은 2000년 발매된 에릭 클랩튼과 B.B....,B.B. 킹,에릭 클랩튼,11,wikipedia
3,282,"2017년 문재인 정부의 정현백 장관은 ""여성혐오에 소극적으로 대처하는 건 더이상 ...",정현백,문재인,11,wikipedia
4,321,요한 바오로 2세는 요제프 라칭거 추기경(훗날의 교황 베네딕토 16세)과 함께 자유...,베네딕토 16세,요한 바오로 2세,11,wikipedia
...,...,...,...,...,...,...
529,32188,그 결과로서 밴스는 존 F. 케네디와 린든 B. 존슨 행정부에서 직위들의 계승에 근...,린든 B. 존슨,존 F. 케네디,11,wikipedia
530,32277,"한편 김대중은 김종필, 박태준과 단일화에 합의하였고, 이에 불만을 품은 신한국당 역...",박태준,김대중,11,wikipedia
531,32284,초반에는 도쿄에서 한 마을인 히나미자와 마을에 전학 온지 얼마 안된 소년 마에바라 ...,마에바라 케이이치,히나미자와,11,wikipedia
532,32328,문 대통령은 이어진 환담 자리에서 싱 대사에게 지난 해 방중 시 시진핑 주석과 리커...,시진핑,리커창,11,wikitree


## 2.1. 커스텀데이터셋 만들기

- **MyDataset**이라는 커스텀 데이터셋을 만들어보고자 한다.
- 구조를 최대한 추상화해보자면 `Iterable[Tuple[Iterable[int] , Iterable[int] , Iterable[int] , int]]`와 같은 구조를 보인다.
- input_ids`(Iterable[int])`, token_type_ids`(Iteralbe[int])`, attention_mask`(Iterable[int])`, lable`(int)`을 튜플로 묶어 Iterable한 구조로 이은 것으로 이해하면 좋을 것 같다.

In [6]:
class MyDataset(torch.utils.data.Dataset ):
    def __init__( self, df : pd.DataFrame,  train_mode :bool = False,) : # 학습용 셋인지 테스트용 셋인지를 판단하는 지표
        assert type(df) == pd.core.frame.DataFrame

        self.train_mode = train_mode
        if train_mode :
            self.class2idx = CLS2IDX
            self.idx2class = IDX2CLS
            # self.labels :Iterable[int] = df['label'].map(lambda x: self.class2idx[x]).values
            self.labels = df['label']
        sentence_list :List[str] = []

        for idx in range(len(df)):
            new_sentence :str = df.loc[idx,'subject_entity'] + "[SEP]"
            new_sentence += df.loc[idx, 'object_entity'] + "[SEP]" 
            new_sentence += df.loc[idx, 'sentence' ]
            sentence_list.append(new_sentence)
        self.sentence_tensor_list :Dict[str , torch.Tensor] = tokenizer(
            sentence_list, 
            padding=True, 
            # truncation=True, 
            return_tensors='pt',
            max_length=512
        )

        
    def __getitem__(self, key :int): # -> Tuple[Any]:
        segment_token_list = []
        segment_token = 0
        for token in self.sentence_tensor_list['input_ids'][key]:
            segment_token_list.append(1 if segment_token==2 else 0)
            if token.item() == 3 and segment_token <3 :
                segment_token += 1
            
        segment_token_list = torch.LongTensor(segment_token_list)

        return (
            self.sentence_tensor_list['input_ids'][key],
            segment_token_list, #self.sentence_tensor_list['token_type_ids'][key], 
            self.sentence_tensor_list['attention_mask'][key],
            self.labels[key] if self.train_mode else None,
        )
    def __len__(self) :
        return len(self.sentence_tensor_list['input_ids'])


- 스페셜 토큰들의 id들을 파악할 수 있다.

In [7]:
print(tokenizer.all_special_ids, '\n', tokenizer.all_special_tokens)

[1, 3, 0, 2, 4] 
 ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


- 간단한 예시를 통해 tokenizer와 MyDataset클래스가 잘 작동하는지 확인해보자

In [8]:
exam_dataset = MyDataset(df=df, train_mode=True)

In [9]:
print(exam_dataset[0][2][:].__len__())
exam_dataset[10][3] # token_type_ids

229


7

In [10]:
tokenizer.decode(exam_dataset[10][0]) #토큰화 -> 인코딩 -> 디코딩 의 절차를 걸친후의 결과

'[CLS] 하비에르 파스토레 [SEP] 아르헨티나 [SEP] 하비에르 파스토레는 아르헨티나 클럽 타예레스의 유소년팀에서 축구를 시작하였다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [11]:
# exam_dataset.idx2class[exam_dataset[10][3]]

In [12]:
#exam_dataset[10][0][:]

## 2.2. 데이터를 분할해주는 함수`(split_data)` 선언하기

In [13]:
def split_data(i) :
    assert type(i) == int
    i = i % 10
    assert 0<=i<=9 #i는 0~9사이의 숫자여야 한다.
    train_data = pd.DataFrame()
    valid_data = pd.DataFrame() 
    for label_idx in range(30):
        batch_len = df_dict_len[label_idx]//10
        train_data_temp = pd.concat([
            df_dict[label_idx][ : i*batch_len ], 
            df_dict[label_idx][(i+1)*batch_len : ],
        ]) # 어떤 클래스의 train 용 데이터
        valid_data_temp = df_dict[label_idx][ i * batch_len : (i+1)*batch_len] # 어떤 클래스의 valid 용 데이터
        train_data = pd.concat([train_data, train_data_temp]).reset_index(drop=True)
        valid_data = pd.concat([valid_data, valid_data_temp]).reset_index(drop=True)
    return MyDataset(train_data, train_mode=True) , MyDataset(valid_data, train_mode=True)

# 3. 학습
## 3.1. auc를 계산해주는 함수`(cal_auc)` 선언하기
- **odds** `(Iterable[Iterable[int]])` :
    - (n, 30) 형태의 행렬이다. 
    - 각 행의 총합은 1이다. 
    - 열의 개수는 분류하고자 하는 클래스의 수를 의미한다.
- **labels** `(Iterable[int])` : 
    - n차원 벡터이다.
    - 각 entry는 정답 label을 의미한다.
- **label_matrix** `(Iterable[Iterable[ 1|0 ]])` :
    - labels를 원핫벡터로 표현한 행렬이다.
    - (n, 30) 형태의 행렬이다.

```python
fpr, tpr, thresholds = roc_curve(
    y_true= label_matrix[:, i] , 
    y_score= odds[:,i],
    pos_label= 1,
)
auc(fpr, tpr)

```
- 위 코드는 label i를 positive class라고 가정했을 때,
- 즉 i를 제외한 다른 class들은 negative class라고 가정했을 때,
- False Positive Rate 대비 True Positive Rate들을 측정하고,
- 이를 통해 label i를 positive라고 둘 때의 auc를 측정하는 코드이다.
- `cal_auc`는 for문을 통해 30개의 label별로 auc를 계산해서 평균를 반환한다.


In [14]:
softmax = torch.nn.Softmax(dim=-1)
def cal_auc(odds, labels):
    odds = np.array(odds)
    labels = np.array(labels)
    #print(odds.shape)
    #print(labels.shape)
    label_matrix = np.zeros([ labels.__len__() , 30 ])
    for row, column in enumerate(labels):
        label_matrix[row, column] = 1
    
    # label_matrix[:,29] 
    auc_list = []
    for i in range(30):
        fpr, tpr, thresholds = roc_curve(
            y_true= label_matrix[:, i] , 
            y_score= odds[:,i],
            pos_label= 1,
        )
        auc_list.append(auc(fpr, tpr))
    auc_score =sum(auc_list)/len(auc_list)
    return auc_score
    

## 3.2. 학습 함수`(lets_train)` 선언하기

In [15]:
def lets_train(
    epoch, 
    BATCH_SIZE, 
    L_RATE,
    LOGGING_POINT,
):
    print("배치사이즈 : ", BATCH_SIZE)
    print("learning_rate : ", L_RATE)
    print("사용된 모델 : ", MODEL_NAME)

    train_dataset, valid_dataset = split_data(epoch)
    
    train_batch = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = BATCH_SIZE,
        shuffle=True,
        drop_last=True,
        num_workers=2
    )

    valid_batch = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size = BATCH_SIZE,
        shuffle=True,
        drop_last=True,
        num_workers=2
    )


    model.train()
    pred_list = []   #List[int]        n차원 벡터
    label_list = []  #List[int]        n차원 벡터
    odd_list = []    #List[List[int]]  (n,30) 행렬, 
                            # 1. 각 행의 합은 1, 
                            # 2.각 n행 m열은 
                            # 3. n번째 데이터의 m번 클래스에 대해 부여한 확률
    accumulattion_label_list = [] #List[int] label_list와 달리 마지막 이터레이션에서만 초기화됨.
    for idx,( input_ids, token_type_ids, attention_mask, label ) in  enumerate( train_batch ) :
        optimizer.zero_grad()
        pred = model(
            input_ids=input_ids.to(device),
            token_type_ids=token_type_ids.to(device),
            attention_mask=attention_mask.to(device),
        ).logits


        # data_batch 묶음에 대해  (batch_size를 n이라고 할 때)
        # 모델이 추정한 label들의 리스트 // n차원 벡터
        pred_list += list(torch.argmax(pred,dim=-1).cpu().numpy()) 
        
        # 모델이 추정한 label별 확률 // (n,30) 행렬  
        odd_list += list(softmax(pred).cpu().detach().numpy())
        # 실제 데이터의 label 리스트 // n차원 벡터
        label_list += list(label.numpy())

        # 지금까지 모델이 추정해온 label들
        accumulattion_label_list  += list(label.numpy())


        loss = criterion(pred, label.to(device))
        if idx%LOGGING_POINT == LOGGING_POINT-1 or idx ==  len(train_batch) - 1 :
            print(
                "loss : ", loss.item(), 
                "\nf1score : {}".format(f1_score(
                    y_true=label_list, y_pred = pred_list, 
                    average ='micro', labels=range(1,30), 
                )),
                "\nacc : {}".format(accuracy_score(
                    y_true=label_list, y_pred = pred_list
                )),
            )
            pred_list =[]   #pred_list와 label_list는
            label_list = [] #매 LOGGING_POINT마다 초기화
        
        # 아래는 auc를 구하는 코드 
        if idx%LOGGING_POINT == LOGGING_POINT-1 or idx ==  len(train_batch) - 1 :
            try : 
                print(f"auc : {cal_auc(odds=odd_list, labels=accumulattion_label_list)}")
            except:
                print(
                    "odd_list = \n", odd_list, 
                    "accumulattion_label_list = \n", accumulattion_label_list 
                )

        if idx ==  len(train_batch) - 1 :
            odd_list = []                   # odd_list와 accumulation_label_list는
            accumulattion_label_list = []   # 마지막 이터레이션에서 초기화
        loss.backward()
        optimizer.step()


    model.eval()
    with torch.no_grad():
        val_pred_list = []
        val_label_list = []
        val_odd_list = []
        val_accumulattion_label_list = []
        for idx, (input_ids, token_type_ids, attention_mask, label) in  enumerate(valid_batch) :
        
            pred = model(
                input_ids=input_ids.to(device),
                token_type_ids=token_type_ids.to(device),
                attention_mask=attention_mask.to(device),
            ).logits

            val_pred_list += list(torch.argmax(pred,dim=-1).cpu().numpy()) 
            val_label_list += list(label.numpy())
            val_odd_list += list(softmax(pred).cpu().detach().numpy())
            val_accumulattion_label_list += list(label.numpy())

            if idx%LOGGING_POINT == LOGGING_POINT-1 or idx ==  len(valid_batch) - 1 :
                print(
                    f"\nval_f1score : {f1_score(y_true=val_label_list, y_pred = val_pred_list, average ='micro', labels=range(1,30))}",
                    f"\nval_acc : {accuracy_score(y_true=val_label_list, y_pred = val_pred_list)}",
                )
                try:
                    print(f"auc : {cal_auc(odds=val_odd_list, labels=val_accumulattion_label_list)}")
                except:
                    print(
                        "val_odd_list = \n", val_odd_list, 
                        "val_accumulattion_label_list = \n", val_accumulattion_label_list ,
                    )
                val_pred_list =[]
                val_label_list = []

            if idx ==  len(valid_batch) - 1 :
                try:
                    print(f"auc : {cal_auc(odds=val_odd_list, labels=val_accumulattion_label_list)}")
                except:
                    print(
                        "val_odd_list = \n", val_odd_list, 
                        "val_accumulattion_label_list = \n", val_accumulattion_label_list ,
                    )
                val_odd_list = []
                val_accumulattion_label_list = []


In [16]:
EPOCH = 10
BATCH_SIZE = 50
L_RATE = 0.000002
optimizer = torch.optim.AdamW(model.parameters(), lr=L_RATE )
criterion = torch.nn.CrossEntropyLoss().to(device)
model.to(device)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
if __name__ == "__main__":
    print(f' 디바이스 : {device} \n 모델 : {MODEL_NAME} {device} \n 에포크 : {EPOCH} \n 배치사이즈 : {BATCH_SIZE} \n 러닝레이트 : {L_RATE}')
    for ep in range(EPOCH):
        print(ep+1, '번째 에폭')
        lets_train(
            epoch = ep, 
            BATCH_SIZE=BATCH_SIZE,
            L_RATE=L_RATE,
            LOGGING_POINT=100,
        )
        # if ep%5 == 4 : 
        #     torch.save(model, f'./cp/2nd_{ep}.pt')

 디바이스 : cuda 
 모델 : kykim/bert-kor-base cuda 
 에포크 : 10 
 배치사이즈 : 50 
 러닝레이트 : 2e-06
1 번째 에폭
배치사이즈 :  50
learning_rate :  2e-06
사용된 모델 :  kykim/bert-kor-base
loss :  2.7382442951202393 
f1score : 0.01711326277865346 
acc : 0.2546
auc : 0.51283514850075


KeyboardInterrupt: 